<a href="https://colab.research.google.com/github/takatakamanbou/Vision/blob/main/Vision2021_ex05c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision2021-ex05c

課題の期限や提出の方法などについては，Visionチーム内に書いてます．

# ニューラルネットやってみよう



## 準備

In [ ]:
import numpy as np

## MNIST の入手

MNIST についてはこちらを参考: http://yann.lecun.com/exdb/mnist/

In [ ]:
# 上記サイトから 4 つのファイルを入手し， gunzip
! for fn in train-images-idx3-ubyte t10k-images-idx3-ubyte train-labels-idx1-ubyte t10k-labels-idx1-ubyte; do if [ ! -e ${fn} ]; then wget -nc http://yann.lecun.com/exdb/mnist/${fn}.gz ; gunzip ${fn}.gz; fi; done
! ls -l

## MNIST を扱う関数の定義と動作確認

In [ ]:
# MNIST のためのクラスの定義

import struct
import os
import numpy as np

class MNIST:
    
    def __init__(self):

        fnImageL = 'train-images-idx3-ubyte'
        fnImageT = 't10k-images-idx3-ubyte'
        fnLabelL = 'train-labels-idx1-ubyte'
        fnLabelT = 't10k-labels-idx1-ubyte'
    
        if not os.path.exists(fnImageL) or not os.path.exists(fnImageT) or not os.path.exists(fnLabelL) or not os.path.exists(fnLabelT):
            print('Please get the MNIST files first.')
            return
       
        self.fnImage = {'L': fnImageL, 'T': fnImageT}
        self.fnLabel  = {'L': fnLabelL, 'T': fnLabelT}
        self.nrow = 28
        self.ncol = 28
        self.nclass = 10
        
        
    def getLabel( self, LT ):
        
        return _readLabel( self.fnLabel[LT] )
 
 
    def getImage( self, LT ):
        
        return _readImage( self.fnImage[LT] )
 
 
##### reading the label file
#
def _readLabel( fnLabel ):
 
    f = open( fnLabel, 'rb' )
 
    ### header (two 4B integers, magic number(2049) & number of items)
    #
    header = f.read( 8 )
    mn, num = struct.unpack( '>2i', header )  # MSB first (bigendian)
    assert mn == 2049
    #print mn, num
 
    ### labels (unsigned byte)
    #
    label = np.array( struct.unpack( '>%dB' % num, f.read() ), dtype = int )
 
    f.close()
 
    return label

 
##### reading the image file
#
def _readImage( fnImage ):
 
    f = open( fnImage, 'rb' )
 
    ### header (four 4B integers, magic number(2051), #images, #rows, and #cols
    #
    header = f.read( 16 )
    mn, num, nrow, ncol = struct.unpack( '>4i', header ) # MSB first (bigendian)
    assert mn == 2051
    #print mn, num, nrow, ncol
 
    ### pixels (unsigned byte)
    #
    npixel = ncol * nrow
    #pixel = np.empty( ( num, npixel ), dtype = int )
    #pixel = np.empty( ( num, npixel ), dtype = np.int32 )
    pixel = np.empty( ( num, npixel ) )
    for i in range( num ):
        buf = struct.unpack( '>%dB' % npixel, f.read( npixel ) )
        pixel[i, :] = np.asarray( buf )
 
    f.close()
 
    return pixel
        

In [ ]:
### MNIST クラスの動作確認 ###

mnist = MNIST()
print(mnist.nrow, mnist.ncol, mnist.nclass)

print( '# MNIST training data' )
dat = mnist.getImage( 'L' )
lab = mnist.getLabel( 'L' )
print( dat.shape, dat.dtype, lab.shape, lab.dtype )
 
print( '# MNIST test data' )
dat = mnist.getImage( 'T' )
lab = mnist.getLabel( 'T' )
print( dat.shape, dat.dtype, lab.shape, lab.dtype )    

In [ ]:
### mini batch indicies for stochastic gradient ascent
#
def makeBatchIndex(N, batchsize):

    idx = np.random.permutation(N)
        
    nbatch = int(np.ceil(float(N) / batchsize))
    idxB = np.zeros(( nbatch, N ), dtype = bool)
    for ib in range(nbatch - 1):
        idxB[ib, idx[ib*batchsize:(ib+1)*batchsize]] = True
    ib = nbatch - 1
    idxB[ib, idx[ib*batchsize:]] = True

    return idxB

## ネットワークの定義（階層型ニューラルネットワーク）

In [ ]:
# 深層学習ライブラリ PyTorch 関係の import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# ネットワークのアーキテクチャを定義するクラス NN 

class NN(nn.Module):

    def __init__(self):
        
        super(NN, self).__init__()
        
        self.isMLP2 = True
        
        if self.isMLP2:
            ### MLP2: 入力 - 中間層 - 出力層
            self.fc1 = nn.Linear(784, 256)   # 入力次元数, 中間層ニューロン数
            self.fc2 = nn.Linear(self.fc1.out_features, 10)  # 出力層ニューロン数
        else:
            ### MLP3: 入力 - 中間層1 - 中間層2 - 出力層
            self.fc1 = nn.Linear(784, 1024)   # 入力次元数, 中間層ニューロン数
            self.fc2 = nn.Linear(self.fc1.out_features, 256)
            self.fc3 = nn.Linear(self.fc2.out_features, 10)  # 出力層ニューロン数
            

    def forward(self, X):
        
        if self.isMLP2:
            X = F.relu(self.fc1(X))
            X = self.fc2(X)
        else:
            X = F.relu(self.fc1(X))
            X = F.relu(self.fc2(X))
            X = self.fc3(X)
        
        
        return F.log_softmax(X, dim = 1)


In [ ]:
# データを与えると損失関数（交差エントロピー）の値と識別率を計算する関数

def evaluate(model, X, Y, bindex):

    nbatch = bindex.shape[0]
    loss = 0
    ncorrect = 0
    with torch.no_grad():
        for ib in range(nbatch):
            ii = np.where(bindex[ib, :])[0]
            output = model(X[ii, ::])
            #loss += F.nll_loss(output, Y[ii], size_average=False).item()
            loss += F.nll_loss(output, Y[ii], reduction='sum').item()
            pred = output.max(1, keepdim=True)[1] # argmax of the output
            ncorrect += pred.eq(Y[ii].view_as(pred)).sum().item()

    loss /= X.shape[0]
    acc = ncorrect / X.shape[0]

    return loss, acc

In [ ]:
# CUDA （NVIDIAのGPGPUライブラリ） が使えるかチェック

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('# using', device)

## 学習データの用意

In [ ]:
# 学習データと検査データ

mn = MNIST()
D = mn.nrow * mn.ncol
K = mn.nclass

datLraw = mn.getImage('L') / 255.0  #  [0, 255] => [0,1]
labLraw = mn.getLabel('L')
datL = datLraw[:50000, :]   # 6万個のうち最初の5万個を学習用に
labL = labLraw[:50000]
datV = datLraw[50000:, :]   # 残りは検査(validation)用に
labV = labLraw[50000:]

NL = datL.shape[0]
NV = datV.shape[0]

### to torch.Tensor
#
XL = torch.from_numpy(datL.astype(np.float32)).to(device)
YL = torch.from_numpy(labL).to(device)
XV = torch.from_numpy(datV.astype(np.float32)).to(device)
YV = torch.from_numpy(labV).to(device)

## 学習させる

In [ ]:
%%time

### initializing the network
#
torch.manual_seed(0)
net = NN()
model = net.to(device)
print(model)
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9)
print(optimizer)

### training
#
batchsize = 128
bindexL = makeBatchIndex(NL, batchsize)
nbatchL = bindexL.shape[0]
bindexV = makeBatchIndex(NV, batchsize)
nbatchV = bindexV.shape[0]

nitr = 10000
nd = 0
    
for i in range(nitr):

    if (i < 500 and i % 100 == 0) or (i % 500 == 0):
            
        model.eval()  # setting the module in evaluation mode
        lossL, accL = evaluate(model, XL, YL, bindexL)
        lossV, accV = evaluate(model, XV, YV, bindexV)
        print(i, nd, end = '     ')
        print('{:.4f} {:.2f}'.format(lossL, accL*100), end = '     ') 
        print('{:.4f} {:.2f}'.format(lossV, accV*100), end = '     ') 
        print()
            
             
    model.train()  # setting the module in training mode
    ib = np.random.randint(0, nbatchL)
    ii = np.where(bindexL[ib, :])[0]
    optimizer.zero_grad()
    output = model(XL[ii, :])
    loss = F.nll_loss(output, YL[ii])
    loss.backward()
    optimizer.step()

    nd += ii.shape[0]

## テストする



In [ ]:
datT = mn.getImage('T') / 255.0  #  [0, 255] => [0,1]
labT = mn.getLabel('T')
NT = datT.shape[0]
XT = torch.from_numpy(datT.astype(np.float32)).to(device)
YT = torch.from_numpy(labT).to(device)
bindexT = makeBatchIndex(NT, batchsize)

model.eval()  # setting the module in evaluation mode
lossT, accT = evaluate(model, XT, YT, bindexT)
print('{:.4f} {:.2f}'.format(lossT, accT*100))